# Style Transfer

<img src="https://i0.wp.com/chelseatroy.com/wp-content/uploads/2018/12/neural_style_transfer.png?resize=768%2C311&ssl=1">

La idea de este trabajo final es reproducir el siguiente paper:

https://arxiv.org/pdf/1508.06576.pdf

El objetivo es transferir el estilo de una imagen dada a otra imagen distinta. 

Como hemos visto en clase, las primeras capas de una red convolucional se activan ante la presencia de ciertos patrones vinculados a detalles muy pequeños.

A medida que avanzamos en las distintas capas de una red neuronal convolucional, los filtros se van activando a medida que detectan patrones de formas cada vez mas complejos.

Lo que propone este paper es asignarle a la activación de las primeras capas de una red neuronal convolucional (por ejemplo VGG19) la definición del estilo y a la activación de las últimas capas de la red neuronal convolucional, la definición del contenido.

La idea de este paper es, a partir de dos imágenes (una que aporte el estilo y otra que aporte el contenido) analizar cómo es la activación de las primeras capas para la imagen que aporta el estilo y cómo es la activación de las últimas capas de la red convolucional para la imagen que aporta el contenido. A partir de esto se intentará sintetizar una imagen que active los filtros de las primeras capas que se activaron con la imagen que aporta el estilo y los filtros de las últimas capas que se activaron con la imagen que aporta el contenido.

A este procedimiento se lo denomina neural style transfer.

# En este trabajo se deberá leer el paper mencionado y en base a ello, entender la implementación que se muestra a continuación y contestar preguntas sobre la misma.

# Una metodología posible es hacer una lectura rápida del paper (aunque esto signifique no entender algunos detalles del mismo) y luego ir analizando el código y respondiendo las preguntas. A medida que se planteen las preguntas, volviendo a leer secciones específicas del paper terminará de entender los detalles que pudieran haber quedado pendientes.

Lo primero que haremos es cargar dos imágenes, una que aporte el estilo y otra que aporte el contenido. A tal fin utilizaremos imágenes disponibles en la web.

In [1]:

%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
# Imagen para estilo
!wget https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg

# Imagen para contenido
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg

#/content/drive/MyDrive/maradona-gano-copa-mundo-mexico.jpg


!wget https://s3.amazonaws.com/arc-wordpress-client-uploads/infobae-wp/wp-content/uploads/2017/04/26120438/Guernica-Pablo-Picasso-pintura-obra-1920-1024x503.jpg

# Creamos el directorio para los archivos de salida
!mkdir /content/output

--2021-01-20 16:19:52--  https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 91.198.174.208, 2620:0:862:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|91.198.174.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 223725 (218K) [image/jpeg]
Saving to: ‘La_noche_estrellada1.jpg.7’

La_noche_estrellada 100%[===================>] 218.48K  --.-KB/s    in 0.02s   

2021-01-20 16:19:52 (13.7 MB/s) - ‘La_noche_estrellada1.jpg.7’ saved [223725/223725]

--2021-01-20 16:19:52--  https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 91.198.174.208, 2620:0:862:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|91.198.174.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 153015 (149K) 

In [3]:
from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse

from keras.applications import vgg19
from keras import backend as K
from pathlib import Path

Using TensorFlow backend.


In [4]:
# Definimos las imagenes que vamos a utilizar, y el directorio de salida

#base_image_path = Path("/content/775px-Neckarfront_Tübingen_Mai_2017.jpg")
base_image_path = Path("/content/drive/MyDrive/maradona-gano-copa-mundo-mexico.jpg")
#style_reference_image_path = Path("/content/La_noche_estrellada1.jpg")
style_reference_image_path = Path("/content/Guernica-Pablo-Picasso-pintura-obra-1920-1024x503.jpg")

result_prefix = Path("/content/output")
iterations = 100

# 1) En base a lo visto en el paper ¿Qué significan los parámetros definidos en la siguiente celda?

Respuesta:

**Style Weight y Content Weight son los factores de peso para la reconstrucción del content y el style. El total_variation_weight es un factor más que se utiliza al final de la función de loss. Los 3 parámetros ajustan la loss. Con estos parámetros se establece el énfasis en el contenido o el estilo que se quiere dar como resultado. Estos parámetros se ven ajustando content_loss y style_loss, por último el total_variation_weigth acelera o desacelera la loss al multiplicarse al final.**


In [5]:
total_variation_weight = 0.1
style_weight = 10
content_weight = 1

In [6]:
# Definimos el tamaño de las imágenes a utilizar
width, height = load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

# 2) Explicar qué hace la siguiente celda. En especial las últimas dos líneas de la función antes del return. ¿Por qué?

Ayuda: https://keras.io/applications/

Respuesta:

**En esta función se carga la imagen a través del path dimensionándola en img_nrows y img_ncols que fueron seteados previamente (en este caso 400 filas y las columnas que den como resultado la misma relación de aspecto).
Luego se pasa la imagen a un numpy array.
Se expande la dimensión para cuando la vgg19 preprocese utilice esta dimensión para determinar la cantidad de imagenes en el batch.
preprocess_input convierte las imagenes de RGB a BGR y las centra en 0**

In [7]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

# 3) Habiendo comprendido lo que hace la celda anterior, explique de manera muy concisa qué hace la siguiente celda. ¿Qué relación tiene con la celda anterior?

Respuesta:

**Esta celda hace el proceso inverso al que se usa en la función anterior. Primero hace el reshape con los 3 canales y filas-columnas que definí al principio. Luego corre cada canal usando la media de las imagenes de ImageNet para que queden entre 0 y 255.
Luego se invierte el orden de las columnas para que quede en RGB y al final se redondea para que queden enteros y dentro del rango.**

In [8]:
def deprocess_image(x):
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [9]:
# get tensor representations of our images
# K.variable convierte un numpy array en un tensor, para 
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [10]:
combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

Aclaración:

La siguiente celda sirve para procesar las tres imagenes (contenido, estilo y salida) en un solo batch.

In [11]:
# combine the 3 images into a single Keras tensor
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

In [12]:
# build the VGG19 network with our 3 images as input
# the model will be loaded with pre-trained ImageNet weights
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)
print('Model loaded.')

# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])


Model loaded.


# 4) En la siguientes celdas:

- ¿Qué es la matriz de Gram?¿Para qué se usa?
- ¿Por qué se permutan las dimensiones de x?

**La matriz de Gram posee la correlación entre las features y el producto punto de las mismas. Captura la distribución de las features. Se utiliza para luego hacer el MSE entre el estilo y la imagen combinada o el contenido y la imagen combinada.
Las dimensiones se permutan para pasar los canales adelante y el alto y el ancho quedan como dimensión 2 y 3**

In [13]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

In [14]:
outputs_dict['block1_conv1'][0]

<tf.Tensor 'strided_slice:0' shape=(400, 642, 64) dtype=float32>

# 5) Losses:

Explicar qué mide cada una de las losses en las siguientes tres celdas.

Rta:
1) **Esta función mide el MSE entre la matriz Gram de las representación del estilo entre la imagen original y la generada. En definitiva, la imagen del estilo generada y la original.**

2) **Esta función mide el MSE entre la imagen generada y la imagen base.**

3) **Esta función sirve para regularizar. Determina cuánto ruido hay en la imagen.**

In [15]:
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

In [16]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))


In [17]:
def total_variation_loss(x):
    assert K.ndim(x) == 4
    a = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
    b = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))


In [18]:
# Armamos la loss total
loss = K.variable(0.0)
layer_features = outputs_dict['block5_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(base_image_features,
                                            combination_features)

feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :] 
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(feature_layers)) * sl
loss = loss + total_variation_weight * total_variation_loss(combination_image)

In [19]:
print(loss)

Tensor("add_7:0", shape=(), dtype=float32)


In [20]:
grads = K.gradients(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [21]:
grads

[<tf.Tensor 'gradients/AddN_16:0' shape=(1, 400, 642, 3) dtype=float32>]

# 6) Explique el propósito de las siguientes tres celdas. ¿Qué hace la función fmin_l_bfgs_b? ¿En qué se diferencia con la implementación del paper? ¿Se puede utilizar alguna alternativa?

Respuesta:

**La función fmin_l_bfgs_b optimiza la minimización de la loss para aproximar parámetros y utilizar menos memoria. En el paper se utiliza gradient descent. Se podría utilizar un optimizador como Adam**

In [22]:
def eval_loss_and_grads(x):
    x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.

In [23]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

# 7) Ejecute la siguiente celda y observe las imágenes de salida en cada iteración.

In [24]:
evaluator = Evaluator()

# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(base_image_path)

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    img = deprocess_image(x.copy())
    fname = result_prefix / ('output_at_iteration_%d.png' % i)
    save_img(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 23369170000.0
Image saved as /content/output/output_at_iteration_0.png
Iteration 0 completed in 7s
Start of iteration 1
Current loss value: 8846557000.0
Image saved as /content/output/output_at_iteration_1.png
Iteration 1 completed in 4s
Start of iteration 2
Current loss value: 6141035500.0
Image saved as /content/output/output_at_iteration_2.png
Iteration 2 completed in 4s
Start of iteration 3
Current loss value: 4806697500.0
Image saved as /content/output/output_at_iteration_3.png
Iteration 3 completed in 4s
Start of iteration 4
Current loss value: 4002313700.0
Image saved as /content/output/output_at_iteration_4.png
Iteration 4 completed in 4s
Start of iteration 5
Current loss value: 3558336500.0
Image saved as /content/output/output_at_iteration_5.png
Iteration 5 completed in 4s
Start of iteration 6
Current loss value: 3201592300.0
Image saved as /content/output/output_at_iteration_6.png
Iteration 6 completed in 4s
Start of iteration 7
Curre

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 8) Generar imágenes para distintas combinaciones de pesos de las losses. Explicar las diferencias. (Adjuntar las imágenes generadas como archivos separados.)

Respuesta:

Realicé varios cambios en los pesos de las losses. Empecé cambiando el regularizador y no obtuve mucho cambio. La imagen 1 es la que venía con los pesos por defecto. La imagen 2 y 3 les modifiqué el peso en el total variation.
Luego en la imagen 4 aumenté el peso en la loss del estilo y el cambio se notó. La imagen se deformó y se parece más a la original del estilo.
En la imagen 5 hice el proceso inverso, y si bien la imagen se modificó. Es mucho más parecida a la original.
Las imágenes las adjunto en una carpeta aparte.
Acá pego los parámetros que usé en cada imagen:
1.png)
total_variation_weight = 0.1
style_weight = 10
content_weight = 1

2.png) 
total_variation_weight = 0.5
style_weight = 10
content_weight = 1

3.png) 
total_variation_weight = 0.0001
style_weight = 10
content_weight = 1


4.png) 
total_variation_weight = 0.1
style_weight = 1000
content_weight = 1

5.png)
total_variation_weight = 0.1
style_weight = 1
content_weight = 100

# 9) Cambiar las imágenes de contenido y estilo por unas elegidas por usted. Adjuntar el resultado.

Respuesta:

Puse una foto de Maradona besando la Copa y la combiné con el cuadro Guernica de Picasso. Me pareció que el estilo blanco y negro y de formas geométricas podía transferirse a la foto original.
Empecé probando con un peso en la loss de estilo bajo, y si bien se transfería el estilo, preferí aumentar ese peso para que se note más.
El resultado fue una imagen con formas geométricas y otras partes que identifico de la imagen original (ver el cuello de Diego) y también el acercamiento al blanco y negro, aunque se perciben algunos colores bastante apagados.
Se adjunta como maradona-picasso.png